In [1]:
!git clone https://github.com/krishnaik06/SpamClassifier.git

Cloning into 'SpamClassifier'...


In [2]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
from nltk import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to C:\Users\Yash A
[nltk_data]     Patel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#!pip freeze requirements.txt
#out = !pip freeze requirements.txt
#with open('requirements.txt', 'w') as writefile:
#    for element in out:
#      writefile.write(element)
#      writefile.write('\n')

absl-py==0.12.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.1.0
appdirs==1.4.4
argon2-cffi==20.1.0
arviz==0.11.2
astor==0.8.1
astropy==4.2.1
astunparse==1.6.3
async-generator==1.10
atari-py==0.2.9
atomicwrites==1.4.0
attrs==21.2.0
audioread==2.1.9
autograd==1.3
Babel==2.9.1
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==3.3.0
blis==0.4.1
bokeh==2.3.2
Bottleneck==1.3.2
branca==0.4.2
bs4==0.0.1
CacheControl==0.12.6
cached-property==1.5.2
cachetools==4.2.2
catalogue==1.0.0
certifi==2020.12.5
cffi==1.14.5
cftime==1.5.0
chainer==7.4.0
chardet==3.0.4
click==7.1.2
cloudpickle==1.3.0
cmake==3.12.0
cmdstanpy==0.9.5
colorcet==2.0.6
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.3.2
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cvxopt==1.2.6
cvxpy==1.0.31
cycler==0.10.0
cymem==2.0.5
Cython==0.29.23
daft==0.0.4
dask==2.12.0
datascience==0.10.6
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.7.1
descartes==1.1.0
dill==0.3.3
distributed==1.25.3
dlib==19.18.

In [8]:
data = pd.read_csv('SpamClassifier/smsspamcollection/SMSSpamCollection', sep='\t',names=['label', 'message'])

In [9]:
data.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [10]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
data['spam'] = labelencoder.fit_transform(data['label'])

In [11]:
data.head()

,label,message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [12]:
data.drop(['label'],inplace=True, axis=1)

In [13]:
import re
ps = PorterStemmer()
wnl = WordNetLemmatizer()
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Yash A
[nltk_data]     Patel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
corpus = []
for message in data['message']:
  final_msg = re.sub('[^a-zA-Z]', ' ', message)
  final_msg = final_msg.lower()
  final_msg = final_msg.split()
  final_msg = [ps.stem(word) for word in final_msg if not word in stopwords.words('english')]
  final_msg = ' '.join(final_msg)
  corpus.append(final_msg)
print(corpus[0:5])

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat', 'ok lar joke wif u oni', 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli', 'u dun say earli hor u c alreadi say', 'nah think goe usf live around though']


In [15]:
from sklearn.feature_extraction.text import CountVectorizer
CV = CountVectorizer()
X = CV.fit_transform(corpus)
y = data['spam']

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [17]:
print(X_train.shape,'   ', X_test.shape,'   ', y_train.shape,'   ', y_test.shape)

(4457, 6296)     (1115, 6296)     (4457,)     (1115,)


In [18]:
from sklearn.naive_bayes import MultinomialNB
spam_detector = MultinomialNB()
spam_detector.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [19]:
y_pred = spam_detector.predict(X_test)
#Accuracy_score = spam_detector.score(y_test, y_pred)

In [20]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [21]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_pred)
print(acc)

0.9748878923766816


In [22]:
import pickle
file = open('spam_model.pkl', 'wb')
pickle.dump(spam_detector, file)

In [23]:
import pickle
file = open('transform.pkl', 'wb')
pickle.dump(CV, file)